In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_jpg.zip', 'train_jpg.zip', 'periods_train.csv', 'train.csv', 'sample_submission.csv', 'train_active.csv', 'test.csv', 'periods_test.csv', 'test_active.csv']


* Explorar, visualizar,
* sklearn preprocess encode
* treino arvore


Lendo CSVs

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
print('Train Columns ',train.columns)
print('Test Columns ',test.columns)

Train Columns  Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')
Test Columns  Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1'],
      dtype='object')


In [4]:
train = train.drop(['image', 'image_top_1'], axis = 1)
test = test.drop(['image', 'image_top_1'], axis = 1)

print ('Train', train)

print ('Test', test)

Train               item_id       ...        deal_probability
0        b912c3c6a6ad       ...                 0.12789
1        2dac0150717d       ...                 0.00000
2        ba83aefab5dc       ...                 0.43177
3        02996f1dd2ea       ...                 0.80323
4        7c90be56d2ab       ...                 0.20797
5        51e0962387f7       ...                 0.80323
6        c4f260a2b48a       ...                 0.00000
7        6b71309d6a8a       ...                 0.80323
8        c5b969cb63a2       ...                 0.00000
9        b1570962e68c       ...                 0.00000
10       d5480bb4a6e4       ...                 0.58853
11       86f41f50d8c1       ...                 0.00000
12       645237cb3601       ...                 0.13858
13       df2116f34563       ...                 0.00000
14       a97943ae8158       ...                 0.00000
15       4b6abd0a5921       ...                 0.00000
16       7896ef8fe482       ...           

In [5]:
print ('Train shape: ',train.shape)
print ('Test shape: ',test.shape)

Train shape:  (1503424, 16)
Test shape:  (508438, 15)


In [6]:
train

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,0.00000


Uma visualização melhor dos dados pode mostrar quais colunas são realmente importantes para o target. Logo de cara, *user_type* ou *user_type* não apresentam, intuitivamente, influência sobre a probabilidade. A Descrição e o título, por sua vez, tratam-se de textos e configuram um cenário que há de ser tratado distintamente dos demais dados categóricos.

Um bom teste seria realizar um treino base com as colunas base.

In [7]:
Y_df = train['deal_probability']
train_df = train.drop(['item_id','activation_date','user_type', 'title','description','deal_probability'], axis = 1)

In [8]:
from sklearn.preprocessing import LabelEncoder

# print (train_df)
for col in train_df.columns:
    print (col)
    enc = LabelEncoder()
    
    train_df[col] = enc.fit_transform(train_df[col].values.astype(str))

user_id
region
city
parent_category_name
category_name
param_1
param_2
param_3
price
item_seq_number


In [9]:
# from sklearn.svm import SVC, SVR

# clf = SVR()
# clf = clf.fit(train_df, Y_df)

In [10]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_df, Y_df)

ValueError: Unknown label type: 'continuous'